In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#change the file location according to your file location in google drive
import os
os.chdir('/content/drive/My Drive/Final_Project_CS498/Filtered_Data_from_Illinois/')

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [4]:
df= pd.read_csv("all_data_with_spd_limt_v2.csv")

In [5]:
df.loc[(df.AccCount > 0) & (df.AccCount<= 30),'Accident']=1
df.loc[(df.AccCount > 30) & (df.AccCount<= 50),'Accident']=2
df.loc[(df.AccCount > 50),'Accident']=3
df.loc[(df.AccCount == 0),'Accident']=0

In [6]:
df.columns

Index(['Unnamed: 0', 'AccCount', 'Severity_Fatal', 'Severity_Inj_TypeA',
       'Severity_Inj_TypeB', 'Severity_Inj_TypeC', 'Severity_PDO',
       'Severity_Unknown', 'Weather_Clear', 'Weather_CrossWind', 'Weather_Fog',
       'Weather_Hail', 'Weather_Other', 'Weather_Rain', 'Weather_Snow',
       'Weather_Unknown', 'Year', 'aadt', 'access', 'begmp', 'cnty_rte',
       'curv_rad', 'curv_rad_binary', 'endmp', 'lanewid', 'med_type',
       'med_type_binary', 'medwid', 'no_lanes', 'oneway', 'rodwycls', 'rururb',
       'seg_lng', 'spd_limt', 'surf_cat', 'surf_typ', 'trf_cntl', 'Accident'],
      dtype='object')

Use only these features

In [7]:
df=df[['aadt',
       'access', 'curv_rad_binary', 
       'lanewid', 'med_type_binary', 'no_lanes',
       'oneway', 'rodwycls', 'rururb', 'seg_lng', 'spd_limt', 'surf_cat', 'Accident']]

In [8]:

# shuffle the DataFrame rows
df = df.sample(frac = 1)
  

In [9]:
df.head()

,aadt,access,curv_rad_binary,lanewid,med_type_binary,no_lanes,oneway,rodwycls,rururb,seg_lng,spd_limt,surf_cat,Accident
556385,2000,0,No Curve,12,No Median,2,2,8,1,0.10,55,Rigid,0.0
626947,3650,0,No Curve,11,No Median,2,2,8,1,0.11,55,Rigid,0.0
665046,1100,0,No Curve,11,No Median,2,2,8,1,0.22,55,Rigid,0.0
431026,15100,0,No Curve,12,No Median,2,2,3,2,0.08,55,Rigid,0.0
315271,1200,0,No Curve,11,No Median,2,2,8,1,0.10,55,Rigid,0.0


In [10]:
df.reset_index(inplace=True)

In [11]:
df.drop(columns='index', inplace=True)

In [12]:
print(df.head())

    aadt  access curv_rad_binary  lanewid  ... seg_lng  spd_limt  surf_cat  Accident
0   2000       0        No Curve       12  ...    0.10        55     Rigid       0.0
1   3650       0        No Curve       11  ...    0.11        55     Rigid       0.0
2   1100       0        No Curve       11  ...    0.22        55     Rigid       0.0
3  15100       0        No Curve       12  ...    0.08        55     Rigid       0.0
4   1200       0        No Curve       11  ...    0.10        55     Rigid       0.0

[5 rows x 13 columns]


In [13]:
test_data=df.iloc[0:2000, :]

In [14]:
print(len(test_data))

2000


In [15]:
train_data=df.iloc[2001:, :]

In [ ]:
#test_data.to_csv("Test_data_selected_features.csv")

In [ ]:
#train_data.to_csv("Train_data_selected_features.csv")

In [16]:
train_data['Accident'].value_counts()

0.0    577713
1.0    247217
2.0      1872
3.0       444
Name: Accident, dtype: int64

In [17]:
train_data=train_data.reset_index()

In [18]:
train_data.head()

,index,aadt,access,curv_rad_binary,lanewid,med_type_binary,no_lanes,oneway,rodwycls,rururb,seg_lng,spd_limt,surf_cat,Accident
0,2001,2100,0,No Curve,13,No Median,2,2,8,1,0.13,55,Rigid,0.0
1,2002,800,0,No Curve,12,No Median,2,2,8,1,0.04,55,Rigid,0.0
2,2003,1500,0,No Curve,11,No Median,2,2,8,1,0.02,55,Rigid,0.0
3,2004,5300,0,No Curve,20,No Median,2,1,99,2,0.06,30,Rigid,1.0
4,2005,15500,2,No Curve,12,Median,4,2,1,2,0.03,65,Rigid,0.0


In [19]:
train_data.drop(inplace= True, columns='index')

Categorical to ordinal

In [20]:
train_data.loc[(train_data.curv_rad_binary == 'No Curve'),'curv_rad_binary']=0
train_data.loc[(train_data.curv_rad_binary == 'Curve'),'curv_rad_binary']=1

In [21]:
train_data['curv_rad_binary']=pd.to_numeric(train_data['curv_rad_binary'])

In [22]:
train_data.loc[(train_data.med_type_binary	 == 'No Median'),'med_type_binary']=0
train_data.loc[(train_data.med_type_binary	 == 'Median'),'med_type_binary']=1

In [23]:
train_data['med_type_binary']=pd.to_numeric(train_data['med_type_binary'])

In [24]:
train_data.loc[(train_data.surf_cat	 == 'Rigid'),'surf_cat']=1
train_data.loc[(train_data.surf_cat	 == 'Flexible'),'surf_cat']=2
train_data.loc[(train_data.surf_cat	 == 'Other'),'surf_cat']=3

In [25]:
train_data['surf_cat']=pd.to_numeric(train_data['surf_cat'])

z_score

In [26]:
# apply normalization technique
column = 'aadt'
train_data[column] = (train_data[column] - train_data[column].mean()) / train_data[column].std()    

In [27]:
print(train_data.rodwycls.value_counts())

8     373993
3     149716
4     110309
5      72613
1      48057
6      35723
9      16349
99     14811
10      2378
7       2344
2        953
Name: rodwycls, dtype: int64


In [28]:
column = 'lanewid'
train_data[column] = (train_data[column] - train_data[column].mean()) / train_data[column].std()

In [ ]:
#column = 'rodwycls'
#train_data[column] = (train_data[column] - train_data[column].mean()) / train_data[column].std()

In [29]:
train_data.head()

,aadt,access,curv_rad_binary,lanewid,med_type_binary,no_lanes,oneway,rodwycls,rururb,seg_lng,spd_limt,surf_cat,Accident
0,-0.481139,0,0,0.727296,0,2,2,8,1,0.13,55,1,0.0
1,-0.535222,0,0,-0.041433,0,2,2,8,1,0.04,55,1,0.0
2,-0.506100,0,0,-0.810162,0,2,2,8,1,0.02,55,1,0.0
3,-0.348013,0,0,6.108399,0,2,1,99,2,0.06,30,1,1.0
4,0.076327,2,0,-0.041433,1,4,2,1,2,0.03,65,1,0.0


In [30]:
def z_score(Column):
  return (train_data[column] - train_data[column].mean()) / train_data[column].std()

In [31]:
train_data['spd_limt']=z_score('spd_limt')

In [32]:
train_data['seg_lng']=z_score('seg_lng')

In [33]:

train_data['no_lanes']=z_score('no_lanes')

In [34]:
train_data.head()

,aadt,access,curv_rad_binary,lanewid,med_type_binary,no_lanes,oneway,rodwycls,rururb,seg_lng,spd_limt,surf_cat,Accident
0,-0.481139,0,0,0.727296,0,0.727296,2,8,1,0.727296,0.727296,1,0.0
1,-0.535222,0,0,-0.041433,0,-0.041433,2,8,1,-0.041433,-0.041433,1,0.0
2,-0.506100,0,0,-0.810162,0,-0.810162,2,8,1,-0.810162,-0.810162,1,0.0
3,-0.348013,0,0,6.108399,0,6.108399,1,99,2,6.108399,6.108399,1,1.0
4,0.076327,2,0,-0.041433,1,-0.041433,2,1,2,-0.041433,-0.041433,1,0.0


features to standardize: 'no_lanes', 'spd_limt', 'seg_lng', 'lanewid', 'rodwycls', 'aadt'.............. when testing also

In [35]:
X= train_data.iloc[:, 0:-1]
y= train_data.iloc[:, -1].values.ravel()

In [37]:
train_data.to_csv("train_data_normalizedv4.csv")

xgboost

In [50]:
model = XGBClassifier(objective='multi:softprob', max_depth=5,  n_estimators=100, 
                        num_classes=4) 

model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, num_classes=4, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

logistic

In [ ]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')

#testing

Categorical to ordinal on test set

In [39]:
test_data.loc[(test_data.curv_rad_binary == 'No Curve'),'curv_rad_binary']=0
test_data.loc[(test_data.curv_rad_binary == 'Curve'),'curv_rad_binary']=1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [40]:
test_data['curv_rad_binary']=pd.to_numeric(test_data['curv_rad_binary'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
test_data.loc[(test_data.med_type_binary	 == 'No Median'),'med_type_binary']=0
test_data.loc[(test_data.med_type_binary	 == 'Median'),'med_type_binary']=1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [42]:
test_data['med_type_binary']=pd.to_numeric(test_data['med_type_binary'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
test_data.loc[(test_data.surf_cat	 == 'Rigid'),'surf_cat']=1
test_data.loc[(test_data.surf_cat	 == 'Flexible'),'surf_cat']=2
test_data.loc[(test_data.surf_cat	 == 'Other'),'surf_cat']=3
test_data['surf_cat']=pd.to_numeric(test_data['surf_cat'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [44]:
test_data.head()

,aadt,access,curv_rad_binary,lanewid,med_type_binary,no_lanes,oneway,rodwycls,rururb,seg_lng,spd_limt,surf_cat,Accident
0,2000,0,0,12,0,2,2,8,1,0.10,55,1,0.0
1,3650,0,0,11,0,2,2,8,1,0.11,55,1,0.0
2,1100,0,0,11,0,2,2,8,1,0.22,55,1,0.0
3,15100,0,0,12,0,2,2,3,2,0.08,55,1,0.0
4,1200,0,0,11,0,2,2,8,1,0.10,55,1,0.0


Do standardization on test data

In [45]:
features_standardize=['no_lanes', 'spd_limt', 'seg_lng', 'lanewid', 'aadt']

In [46]:
for feature in features_standardize:
  test_data[feature]=z_score(feature)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
test_data.head()

,aadt,access,curv_rad_binary,lanewid,med_type_binary,no_lanes,oneway,rodwycls,rururb,seg_lng,spd_limt,surf_cat,Accident
0,0.727296,0,0,0.727296,0,0.727296,2,8,1,0.727296,0.727296,1,0.0
1,-0.041433,0,0,-0.041433,0,-0.041433,2,8,1,-0.041433,-0.041433,1,0.0
2,-0.810162,0,0,-0.810162,0,-0.810162,2,8,1,-0.810162,-0.810162,1,0.0
3,6.108399,0,0,6.108399,0,6.108399,2,3,2,6.108399,6.108399,1,0.0
4,-0.041433,0,0,-0.041433,0,-0.041433,2,8,1,-0.041433,-0.041433,1,0.0


In [48]:
X_test=test_data.iloc[:, 0:-1]
y_test=test_data.iloc[:, -1]

In [51]:
y_pred = model.predict(X_test) 

In [52]:
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 64.55%


In [49]:
test_data.to_csv("test_normalized_v4.csv")